In [3]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 15.1 MB/s eta

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're usin

RuntimeError: ignored

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="l7JkKRdzB4j8haovrhMR")
project = rf.workspace("researchexperiment2").project("aidem_ds")
dataset = project.version(1).download("voc")

In [5]:
!unzip -q /content/gdrive/Shareddrives/research/mosquito_early_prediction.v1i.voc.zip

In [6]:
train_data = object_detector.DataLoader.from_pascal_voc(
    images_dir='/content/mosquito_early_prediction.v1i.voc/train',
    annotations_dir='/content/mosquito_early_prediction.v1i.voc/train',
    label_map=['aedes','non_aedes']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    images_dir='/content/mosquito_early_prediction.v1i.voc/valid',
    annotations_dir='/content/mosquito_early_prediction.v1i.voc/valid',
    label_map=['aedes','non_aedes']
)

In [7]:
spec = model_spec.get('efficientdet_lite1')

In [9]:
model = object_detector.create(train_data, model_spec=spec, batch_size=32, train_whole_model=True, validation_data=val_data, epochs=100)

Epoch 1/100
277/277 [==============================] - 214s 647ms/step - det_loss: 0.7539 - cls_loss: 0.4746 - box_loss: 0.0056 - reg_l2_loss: 0.0709 - loss: 0.8248 - learning_rate: 0.0240 - gradient_norm: 1.9248 - val_det_loss: 1.0568 - val_cls_loss: 0.6686 - val_box_loss: 0.0078 - val_reg_l2_loss: 0.0716 - val_loss: 1.1284
Epoch 2/100
277/277 [==============================] - 176s 635ms/step - det_loss: 0.4792 - cls_loss: 0.2935 - box_loss: 0.0037 - reg_l2_loss: 0.0724 - loss: 0.5516 - learning_rate: 0.0400 - gradient_norm: 1.5787 - val_det_loss: 0.4879 - val_cls_loss: 0.2941 - val_box_loss: 0.0039 - val_reg_l2_loss: 0.0730 - val_loss: 0.5609
Epoch 3/100
277/277 [==============================] - 178s 641ms/step - det_loss: 0.4193 - cls_loss: 0.2590 - box_loss: 0.0032 - reg_l2_loss: 0.0733 - loss: 0.4926 - learning_rate: 0.0399 - gradient_norm: 1.3561 - val_det_loss: 0.6142 - val_cls_loss: 0.4255 - val_box_loss: 0.0038 - val_reg_l2_loss: 0.0737 - val_loss: 0.6879
Epoch 4/100
277/277

In [10]:
model.export(export_dir='/content/gdrive/Shareddrives/research', tflite_filename='aidem101.tflite')

In [11]:
!tensorboard --logdir training_events_path


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)


In [12]:
model.evaluate(val_data)

17/17 [==============================] - 25s 817ms/step



{'AP': 0.79759824,
 'AP50': 0.9352128,
 'AP75': 0.867051,
 'APs': -1.0,
 'APm': 0.9,
 'APl': 0.7980107,
 'ARmax1': 0.7179669,
 'ARmax10': 0.87445045,
 'ARmax100': 0.879689,
 'ARs': -1.0,
 'ARm': 0.9,
 'ARl': 0.87969184,
 'AP_/aedes': 0.7464647,
 'AP_/non_aedes': 0.8487318}

In [15]:
test_data = object_detector.DataLoader.from_pascal_voc(
    images_dir='/content/mosquito_early_prediction.v1i.voc/test',
    annotations_dir='/content/mosquito_early_prediction.v1i.voc/test',
    label_map=['aedes','non_aedes']
)

In [16]:
model.evaluate_tflite('/content/gdrive/Shareddrives/research/aidem101.tflite',test_data)

1033/1033 [==============================] - 4851s 5s/step



{'AP': 0.77562106,
 'AP50': 0.94226974,
 'AP75': 0.85294676,
 'APs': -1.0,
 'APm': 0.03952381,
 'APl': 0.77622235,
 'ARmax1': 0.7600954,
 'ARmax10': 0.8414461,
 'ARmax100': 0.84267664,
 'ARs': -1.0,
 'ARm': 0.5,
 'ARl': 0.84295774,
 'AP_/aedes': 0.7289994,
 'AP_/non_aedes': 0.82224274}

In [20]:
print(model.summary)

<bound method CustomModel.summary of <tensorflow_examples.lite.model_maker.core.task.object_detector.ObjectDetector object at 0x7feaa50ecaf0>>
